# Training `keras_model` with different configurations

- a `keras_model` was created
    + A Deep Convolutional Neural Network inspired by VGG-Net,
    + which takes input of shape `input_shape=(None, 21, 64, 1)`, and
    + produces ___softmax___ outputs for `nclasses=3`:
        * `0`: no speakers active (silence) (, or non-speech)
        * `1`: single speaker active
        * `2`: more than one speaker active
    + The model has been created and compiled by calling `rennet.utils.keras_utils.model_c3(input_shape, nclasses)`
    + having $> 600,000$ trainable parameters


- a python script `train_eval.py` for training and evaluating the model for different configurations was created in `scripts/fisher`
    + the extracted features are read from the exported `trn.h5` (training), `val.h5` (validation) and `tst.h5` (testing), from the directory set by the environment variable `PICKLES_DIR`.
    + the configuration is parsed and set by reading the environment variable `ACTIVITY_NAME`, as:
        1. `no-n/keepzero` :: No normalization, keep silence
        1. `no-n/skipzero` :: No normalization, skip silence **(predictions still for 3 classes, not trained for silence class)**
        1. `no-n/skipzero-20one` :: No normalization, skip silence, sub-sample single speaker class to `20%`, randomly.
        1. `m-n/keepzero` :: Mean normalization, keep silence
        1. `m-n/skipzero` :: Mean normalization, skip silence
        1. `m-n/skipzero-20one` :: Mean normalization, skip silence, sub-sample single speaker class to `20%`, randomly.
        1. `mv-n/keepzero` :: Mean + Variance normalization, keep silence
        1. `mv-n/skipzero` :: Mean + Variance normalization, skip silence
        1. `mv-n/skipzero-20one` :: Mean + Variance normalization, skip silence, sub-sample single speaker class to `20%`, randomly.
    + the output logs, model checkpoints, confusion matrices, etc. are exported to the output path set by the environment variable `ACTIVITY_OUT_DIR`.
        * A `keras_model` checkpoint (containing both the model and the weights) will be created after **every** `keras_epoch`, having the validation loss and `categorical_accuracy` in their names.
        * `logs.txt` has keras's outputs while training, and lines having `P(REC)` with _classwise_ `{precision}({recall})` scores on the validation data after every `keras_epoch`. At the end, the last model is used to evaluate on the testing set, printing the final confusion matrices for raw and Viterbi Smoothed predictions.
        * A file with data for Tensorboard will also be created, updated for each `keras_epoch`.
        * `confs.h5` will be created to store the predictions and confusion matrices for each `keras_epoch` on the validation set used.
        * `confs.test.h5` will be created to store the confusion matrices for all calls, and total confusion matrix on the testing set, at the end, using the last `keras_model`.
    + Various parameters for training on different configurations are **hard-coded** in the script
        * (`trn_callids`, `val_callids`, `tst_callids`) determine which `callids` are used for their (respective) purpose(s) of (training, validation, testing) from the (`trn.h5`, `val.h5`, `tst.h5`).
        * An `epochs_per_pass` number of `keras_epoch`s are run for each pass over the training dataset used.
        * A `steps_per_chunk` number of (shuffled) steps are taken per chunk read from the training data, to avoid exploding the memory. 
        * Viterbi smoothing is _only_ performed when evaluating the testing set at the end, and **not** during validation every epoch to reduce training time.


- `scripts/common.sh` was created to set important, common, environment variables
    + **Update the variables for your specific settings. See README.md**
- `qsub` submittable `bash` scripts were created for ___each___ configuration
    + It will setup the `qsub` parameters as `#PBS` commands at the top of the script.
    + It will activate the training virtual environment and setup the environment variables required by `train_eval.py`, if not already set in `common.sh`.
